# Regression model loading

In [9]:
from tensorflow.keras.models import model_from_json
import numpy as np

with open("./_models/valence_arousal_decomposition/model.json", 'r') as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
model.load_weights("./_models/valence_arousal_decomposition/saved_weights")

# Set up emotion values

In [10]:
VA_emotions = {
    'anger':    [ 1.5, 6.72],   # гнев
    'contempt': [ 1.5, 3.75],   # презрение
    'disgust':  [ 3.5,  2.5],   # отвращение
    'fear':     [ 2.0,  6.2],   # страх
    'happy':    [ 6.6,  4.2],   # радость
    'neutral':  [ 4.0,  4.0],   # безразличие
    'sad':      [ 1.5,  1.5],   # грусть
    'surprise': [ 6.0,  6.0],   # удивление
    'uncertain':[ 3.0,  4.0]    # неопределенность
}

# Prediction for each file in test dataset

In [11]:
import os, glob, pandas as pd
from scipy import spatial

filenames, emotions = [], []

# I use K-dimensional Tree from scipy to find closest emotion vector for model prediction
emotions_vectors = spatial.KDTree(list(VA_emotions.values()))

from PIL import Image
for filename in glob.glob('./_datasets/test_dataset/*.jpg'):
  with open(filename, 'r') as f:
    img = Image.open(filename)
    img = img.resize((224,224))
    img = np.array(img)
    img = np.expand_dims(img, 0)
    prediction = model.predict(img)
    emotion_number = int(emotions_vectors.query(prediction)[1])
    emotion_name = list(VA_emotions.keys())[emotion_number]
    filenames.append(filename)
    emotions.append(emotion_name)

# Create dataframe

In [12]:
data = {'image_path': filenames, 'emotion': emotions}
df = pd.DataFrame(data)
df['image_path'] = df['image_path'].apply(lambda x: x[25:])
df.head()

,image_path,emotion
0,0.jpg,contempt
1,1.jpg,neutral
2,10.jpg,uncertain
3,100.jpg,neutral
4,1000.jpg,happy


# Save submission file

In [13]:
df.to_csv('VA_submission.csv', index=False)